In [ ]:
import os
import pandas as pd
from binance.client import Client
import math
from pyalgotrading.utils.candlesticks import heikinashi
from IPython.display import clear_output
import time
import sys
import datetime

In [ ]:
class TradeBot:
    def __init__(self,user_key,secret_key,symbol,leverage):
        self.user_key=user_key
        self.secret_key=secret_key
        self.binance_client=Client(user_key, secret_key)
        self.symbol=symbol
        self.leverage=leverage
        self.binance_client.futures_change_leverage(symbol=self.symbol, leverage=self.leverage)
        
    def UP_BUY(self,Quantity):
        self.binance_client.futures_create_order(
        symbol=self.symbol,
        type='MARKET',
        side='BUY',
        quantity=Quantity
        )
    def UP_SELL(self,Quantity):
        self.binance_client.futures_create_order(
        symbol=self.symbol,
        type='MARKET',
        side='SELL',
        quantity=Quantity
        )
    def DOWN_BUY(self,Quantity):
        self.binance_client.futures_create_order(
        symbol=self.symbol,
        type='MARKET',
        side='SELL',
        quantity=Quantity
        )
    def DOWN_SELL(self,Quantity):
        self.binance_client.futures_create_order(
        symbol=self.symbol,
        type='MARKET',
        side='BUY',
        quantity=Quantity
        )
    def CurrentCoinPrice(self):
        return float(self.binance_client.get_symbol_ticker(symbol=self.symbol)['price'])
        
    def Quantity(self):
        account_balance=float(self.binance_client.futures_account_balance()[1]['balance'])
        coin_price=float(self.binance_client.get_symbol_ticker(symbol=self.symbol)['price'])
        quantity_to_be_purchased=math.floor(account_balance/coin_price)*leverage
        return quantity_to_be_purchased,coin_price

    def compute_HA(self,interval):
        prev_date=datetime.datetime.utcnow().date()-datetime.timedelta(days=1)
        df = pd.DataFrame(self.binance_client.futures_historical_klines(
             symbol=self.symbol,
             interval=interval,
             start_str=str(prev_date)
        ))
        df = df.iloc[:, :6]
        # ascribe names to columns
        df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
        # convert timestamp to date format and ensure ohlcv are all numeric
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        for col in df.columns[1:]:
            df[col] = pd.to_numeric(df[col])

        df_ha=heikinashi.HeikinAshi(japanese_data=df)
        return df_ha
    def isUP(self,interval='1h'):
        df=self.compute_HA(interval)
        open_price=df.iloc[-1]['open']
        close_price=df.iloc[-1]['close']
        if close_price>=open_price:
            return True
        return False
    
    def CandleStickMean(self,interval='15m'):
        df=self.compute_HA(interval)
        df['CandleLength']=df['close']-df['open']
        positive=df[df['CandleLength']>=0]
        negative=df[df['CandleLength']<0]
        pos=positive['CandleLength'].mean()
        neg=-negative['CandleLength'].mean()
        return pos,neg
    
    def CurrentCandleStick(self,interval='15m'):
        df=self.compute_HA(interval)
        open_price=df.iloc[-1]['open']
        close_price=df.iloc[-1]['close']
        difference=close_price-open_price
        if difference<0:
            difference=-difference
        return difference 
    
    def CurrentTimeStamp(self,interval='15m'):
        df=self.compute_HA(interval)
        TimeStamp=df.iloc[-1]['timestamp']
        return TimeStamp
    
    def DownOrUp(self,interval='15m'):
        df=self.compute_HA(interval)
        open_price=df.iloc[-1]['open']
        close_price=df.iloc[-1]['close']
        return open_price ,close_price
    
    def isHoldingPosition(self):
        for x in self.binance_client.futures_position_information():
            if x['symbol']==self.symbol:
                if x['positionAmt']!='0.0':
                    return True
        return False
        
    def reUPBuy(self):
        while(1):
            try:
                if self.isHoldingPosition():
                    print(12)
                    break
                order_quantity,coin_price=self.Quantity()
                print('Trying to buy UP at : ',coin_price,' Quantity: ',order_quantity)
                    
                self.UP_BUY(order_quantity)
                break
            except:
                print('Whew!', sys.exc_info()[0], 'occurred.')
                clear_output(wait=True)
        
        return order_quantity,coin_price
                            
    def reDOWNBuy(self):
        
        while(1):
            try:
                if self.isHoldingPosition():
                    print(1)
                    break
                order_quantity,coin_price=self.Quantity()
                print('Trying to buy down at : ',coin_price,' Quantity: ',order_quantity)
                
                self.DOWN_BUY(order_quantity)
                break
            except:
                print('Whew!', sys.exc_info()[0], 'occurred.')
                clear_output(wait=True)
        
        return order_quantity,coin_price
    
    def CalculateProfitLoss(self,coin_price,currentcoinprice):
        return (((currentcoinprice/coin_price))-1)*100*self.leverage
                 
                
    def Run15mBot(self):
        try:
            Dict10Percent={}
            while(1):
                if self.CurrentTimeStamp() not in Dict10Percent.keys():
                    if self.isUP('15m'):
                        pos,neg=self.CandleStickMean()
                        pos=pos/4
                        if self.CurrentCandleStick()>pos:
                            order_quantity,coin_price=self.Quantity()
                            timeStamp=self.CurrentTimeStamp()
                            try:
                                Dict10Percent[timeStamp]=[False,{'PosNegPercentage':0}]
                                self.UP_BUY(order_quantity)
                            except:
                                order_quantity,coin_price=self.reUPBuy()                        

                            while(1):
                                if self.isUP('15m'):
                                    currentcoinprice=self.CurrentCoinPrice()
                                    Dict10Percent[timeStamp][1]['PosNegPercentage']=self.CalculateProfitLoss(coin_price,currentcoinprice)
                                    if Dict10Percent[timeStamp][1]['PosNegPercentage']>=10:
                                        Dict10Percent[timeStamp][0]=True

                                    if Dict10Percent[timeStamp][0]:
                                        if Dict10Percent[timeStamp][1]['PosNegPercentage']<10:
                                            self.UP_SELL(order_quantity)
                                            Dict10Percent[self.CurrentTimeStamp()]='Sold'
                                            break


                                    if currentcoinprice>=coin_price:
                                        profit=(currentcoinprice*order_quantity)-(coin_price*order_quantity)
                                        print('Profit = ',profit,' Percentage = ',Dict10Percent[timeStamp][1]['PosNegPercentage'])
                                    else:
                                        loss=(currentcoinprice*order_quantity)-(coin_price*order_quantity)
                                        print('Loss = ',loss,' Percentage = ',Dict10Percent[timeStamp][1]['PosNegPercentage'])
                                    clear_output(wait=True)
                                else:
                                    self.UP_SELL(order_quantity)
                                    Dict10Percent[self.CurrentTimeStamp()]='Sold'
                                    break
                        else:
                            print('Consolidating')
                            clear_output(wait=True)
                    else :
                        pos,neg=self.CandleStickMean()
                        neg=neg/4
                        if self.CurrentCandleStick()>neg:
                            order_quantity,coin_price=self.Quantity()
                            try:
                                Dict10Percent[timeStamp]=[False,{'PosNegPercentage':0}]
                                self.DOWN_BUY(order_quantity)
                            except:
                                order_quantity,coin_price=self.reDOWNBuy()

                            while(1):
                                if (not self.isUP('15m')):
                                    currentcoinprice=self.CurrentCoinPrice()
                                    Dict10Percent[timeStamp][1]['PosNegPercentage']=self.CalculateProfitLoss(currentcoinprice,coin_price)
                                    
                                    if Dict10Percent[timeStamp][1]['PosNegPercentage']>=10:
                                        Dict10Percent[timeStamp][0]=True

                                    if Dict10Percent[timeStamp][0]:
                                        if Dict10Percent[timeStamp][1]['PosNegPercentage']<10:
                                            self.DOWN_SELL(order_quantity)
                                            Dict10Percent[self.CurrentTimeStamp()]='Sold'
                                            break
                                            
                                    if currentcoinprice<=coin_price:
                                        profit=(coin_price*order_quantity)-(currentcoinprice*order_quantity)
                                        print('Profit = ',profit,' Percentage = ',Dict10Percent[timeStamp][1]['PosNegPercentage'])
                                    else:
                                        loss=(coin_price*order_quantity)-(currentcoinprice*order_quantity)
                                        print('Loss = ',loss,' Percentage = ',Dict10Percent[timeStamp][1]['PosNegPercentage'])
                                    clear_output(wait=True)
                                else:
                                    self.DOWN_SELL(order_quantity)
                                    Dict10Percent[self.CurrentTimeStamp()]='Sold'
                                    break
                        else:
                            print('Consolidating')
                            clear_output(wait=True)
                    
                    
                else:
                    print('Checking Entry Point')
                    clear_output(wait=True)

        except:
            print('Whew!', sys.exc_info()[0], 'occurred.')
            clear_output(wait=True)
            self.Run15mBot()
                
                

                
                


In [ ]:
user_key=
secret_key=
symbol='XRPUSDT'
leverage=10

In [ ]:
TB=TradeBot(user_key,secret_key,symbol,leverage)

In [ ]:
TB.Run15mBot()

In [ ]:
binance=Client(user_key,secret_key)